In [99]:
import numpy as np
from sklearn.preprocessing import StandardScaler # Стандартзация данных
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering # Кластеризация
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, homogeneity_score, completeness_score, adjusted_rand_score

### Задание 4.1
Так как изначально данные были представлены для решения задачи классификации, то они находятся в разных файлах (обучающая и тестовая выборки в соотношении 70/30). Соедините признаки так, чтобы сначала шла обучающая выборка, а затем — тестовая, и отдельно соедините значения целевых переменных (разумеется, в том же порядке).

Какая размерность получилась у набора данных с признаками?

In [34]:
X_train = np.loadtxt("../../data/Physical Activity Data/train.txt")
y_train = np.loadtxt("../../data/Physical Activity Data/train_labels.txt")
 
X_test = np.loadtxt("../../data/Physical Activity Data/test.txt")
y_test = np.loadtxt("../../data/Physical Activity Data/test_labels.txt")

X=np.concatenate((X_train, X_test))
y=np.concatenate((y_train, y_test))
print('Количество объектов: ', X.shape[0])
print('Количество признаков: ', X.shape[1])

Количество объектов:  10299
Количество признаков:  561


### Задание 4.2
Теперь найдите число различных активностей, то есть на сколько кластеров в идеале должны разделиться наблюдения.

In [44]:
np.unique(y).shape[0]

6

Как видите, есть ряд активностей, обозначенных цифрами. Эти метки означают следующее:
* 1 — ходьба;
* 2 — подъём;
* 3 — спуск;
* 4 — сидение;
* 5 — стояние;
* 6 — лежание.

### Задание 4.3
Далее необходимо отмасштабировать признаки. Будем использовать для этого алгоритм StandardScaler. Примените его ко всем значениям признаков и впишите в качестве ответа значение первого признака для первого объекта, предварительно округлив его до двух знаков после точки-разделителя.

In [46]:
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
display(scaler)
print(round(X_scaler[0,0],2), ' самое первое значение в самой первой строке')

StandardScaler()

0.21  самое первое значение в самой первой строке


### Задание 4.4
Пора переходить к кластеризации. Для начала определите оптимальное количество кластеров, используя внутренние меры кластеризации. Используйте все известные вам коэффициенты, реализуемые в библиотеке sklearn: коэффициент силуэта, индекс Калински — Харабаса и индекс Дэвиса — Болдина. В качестве алгоритма возьмите k-means++, в качестве значения параметра random_state — число 42.

Выведите оптимальное количество кластеров для каждой метрики, перебирая значения от 2 до 9 включительно. Также введите значение каждой метрики, округлённое до двух знаков после точки-разделителя.

In [56]:
# создадим пустой словарь, ключами будут инерция и количество кластеров
res = {"Количество_кластеров": [], "силуэт": [], "Калински_Харабаса": [], "Дэвиса_Болдина": []}
for i in range(2, 10):
    model =  KMeans(n_clusters=i, init='k-means++', random_state=42) # ( init='k-means++', n_init=10)
    model.fit(X_scaler)
    res["Количество_кластеров"].append(i)
    res["силуэт"].append(silhouette_score(X_scaler,  model.labels_, metric='euclidean'))
    res["Калински_Харабаса"].append(calinski_harabasz_score(X_scaler, model.labels_))
    res["Дэвиса_Болдина"].append(davies_bouldin_score(X_scaler, model.labels_))
    
df=pd.DataFrame(res)
display(df.sort_values(by='силуэт', ascending=False).head(1))
display(df.sort_values(by='Калински_Харабаса', ascending=False).head(1))
display(df.sort_values(by='Дэвиса_Болдина', ascending=True).head(1))

c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set th

,Количество_кластеров,силуэт,Калински_Харабаса,Дэвиса_Болдина
0,2,0.393732,7880.813904,1.070744


,Количество_кластеров,силуэт,Калински_Харабаса,Дэвиса_Болдина
0,2,0.393732,7880.813904,1.070744


,Количество_кластеров,силуэт,Калински_Харабаса,Дэвиса_Болдина
0,2,0.393732,7880.813904,1.070744


### Задание 4.5
Теперь давайте оценим качество кластеризации с точки зрения внешних мер. Реализуйте кластеризацию с помощью классической версии алгоритма k-means. Пусть количество кластеров будет соответствовать количеству активностей. Задайте следующие значения параметров: random_state=42, init='random'.

В качестве ответов введите значения получившихся мер, предварительно округлив их до двух знаков после точки-разделителя.

In [62]:
model =  KMeans(n_clusters=6, init='random', random_state=42)
y_predict = model.fit_predict(X_scaler)
print('Однородность: {:.2f}'.format( homogeneity_score(y, y_predict)))
print('Полнота: {:.2f}'.format( completeness_score(y, y_predict)))
print('ARI (скорректированный индекс Рэнда): {:.2f}'.format( adjusted_rand_score(y, y_predict)))

c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Однородность: 0.54
Полнота: 0.58
ARI (скорректированный индекс Рэнда): 0.42


### Задание 4.6
Выясните, к каким кластерам были преимущественно отнесены различные активности (т. е. в какой кластер попало большинство наблюдений с этой активностью). Нумерацию кластеров начинайте с 1.

In [95]:
df=pd.DataFrame({'y':y, 'predict':y_predict+1, '1':1})
df.pivot_table(index='y', aggfunc='count')
# df.groupby(['y', 'predict']).count()
df.pivot_table(index='y',columns='predict', values='1',aggfunc='count')

predict,1,2,3,4,5,6
y,,,,,,
1.0,NaN,NaN,903.0,78.0,741.0,NaN
2.0,NaN,NaN,1242.0,5.0,295.0,2.0
3.0,NaN,NaN,321.0,196.0,889.0,NaN
4.0,91.0,1238.0,1.0,NaN,NaN,447.0
5.0,NaN,1346.0,NaN,NaN,NaN,560.0
6.0,1556.0,54.0,5.0,NaN,NaN,329.0


### Задание 4.7
Теперь попробуйте реализовать алгоритм k-means для двух кластеров (для того числа активностей, которое является оптимальным с точки зрения внутренних мер) и снова посмотреть, как алгоритм разобьёт активности по кластерам. Как и в предыдущем задании, нумерацию кластеров начинайте с 1.

In [96]:
model =  KMeans(n_clusters=2, init='random', random_state=42)
y_predict = model.fit_predict(X_scaler)
df=pd.DataFrame({'y':y, 'predict':y_predict+1, '1':1})
df.pivot_table(index='y',columns='predict', values='1',aggfunc='count')

c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


predict,1,2
y,,
1.0,1722.0,NaN
2.0,1536.0,8.0
3.0,1406.0,NaN
4.0,3.0,1774.0
5.0,NaN,1906.0
6.0,12.0,1932.0


### Задание 4.8
Вычислите значение полноты для разбиения на два кластера алгоритмом k-means. Ответ округлите до двух знаков после точки-разделителя.

In [97]:
print('Полнота: {:.2f}'.format( completeness_score(y, y_predict)))

Полнота: 0.98


### Задание 4.10
Давайте сравним полученный результат с агломеративной иерархической кластеризацией. Реализуйте её также для двух кластеров и вычислите значение полноты.

In [100]:
model =  AgglomerativeClustering(n_clusters=2, affinity='euclidean')
y_predict = model.fit_predict(X_scaler)
print('Полнота: {:.2f}'.format( completeness_score(y, y_predict)))

c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


Полнота: 1.00


```python
model =  AgglomerativeClustering(n_clusters=6, affinity='euclidean')
y_predict = model.fit_predict(X_scaler)
print('Полнота: {:.2f}'.format( completeness_score(y, y_predict)))
# 0.64
```